In [ ]:
# ---------------------------
# 1) 라이브러리 설치 & import
# ---------------------------

# (이미 설치된 경우 생략 가능)
# !pip install langchain langchain_teddynote chromadb langgraph python-dotenv transformers torch langchain_community pypdf

import os
from dotenv import load_dotenv

# .env 파일에서 OPENAI_API_KEY, LANGCHAIN_API_KEY 등을 로드
load_dotenv()

from core.config import settings

print("OPENAI_API_KEY:", settings.OPENAI_API_KEY)
print("DB_PATH:", settings.DB_PATH)
print("EMBEDDING_MODEL:", settings.EMBEDDING_MODEL)

print("필요 라이브러리 import 및 Settings 로딩 완료")

In [23]:
from service.vectorstore import get_vectorstore
from service.retriever import get_self_query_retriever
from service.reranker import get_compression_retriever
from service.chain import get_rag_chain, rag_pipeline
from typing import TypedDict, Annotated, List

# LangGraph
from langgraph.graph import StateGraph, START, END
from langchain_teddynote.graphs import visualize_graph


class State(TypedDict):
    query: Annotated[str, "User query"]
    vectorstore_built: bool
    retriever_built: bool
    reranker_built: bool
    chain_built: bool
    answer: str

In [ ]:
def ask_user_query(state: State) -> State:
    """노드1: 사용자 질의 입력"""
    user_input = input("자취방 관련 질문을 입력하세요: ")
    state["query"] = user_input
    print(f"[DEBUG] 입력된 질문: {user_input}")
    return state


def build_vectorstore_node(state: State) -> State:
    """노드2: get_vectorstore() 호출 (모듈화 시연)"""
    vs = get_vectorstore()  # 실제로 vs 객체는 내부적으로 캐싱/사용 가능
    state["vectorstore_built"] = True
    print("[DEBUG] VectorStore 빌드 완료")
    return state


def build_retriever_node(state: State) -> State:
    """노드3: get_self_query_retriever() 호출 (모듈화 시연)"""
    retriever = get_self_query_retriever()
    state["retriever_built"] = True
    print("[DEBUG] SelfQueryRetriever 빌드 완료")
    return state


def build_reranker_node(state: State) -> State:
    """노드4: get_compression_retriever(base_retriever) 호출 (모듈화 시연)"""
    # 실제로는 base_retriever가 필요하지만, 여기선 '데모'이므로 호출만 시연
    # base_retriever = get_self_query_retriever()
    # reranker = get_compression_retriever(base_retriever, top_n=4)
    state["reranker_built"] = True
    print("[DEBUG] CrossEncoderReranker 빌드 완료")
    return state


def build_chain_node(state: State) -> State:
    """노드5: get_rag_chain() 호출"""
    # 내부적으로 get_self_query_retriever()를 다시 호출...
    chain = get_rag_chain()
    state["chain_built"] = True
    print("[DEBUG] RAG Chain 빌드 완료")
    return state


def run_pipeline_node(state: State) -> State:
    """노드6: rag_pipeline(question)을 실제로 호출"""
    question = state["query"]
    answer = rag_pipeline(question)
    state["answer"] = answer
    print("[DEBUG] rag_pipeline 실행 완료")
    return state


def show_answer_node(state: State) -> State:
    """노드7: 최종 답변 출력"""
    print("\n=== 자취방 RAG 최종 답변 ===")
    print(state["answer"])
    print("================================\n")
    return state


print("노드 함수(7개) 정의 완료")

In [ ]:
workflow = StateGraph(State)

# 노드 등록
workflow.add_node("ask_user_query", ask_user_query)
workflow.add_node("build_vectorstore", build_vectorstore_node)
workflow.add_node("build_retriever", build_retriever_node)
workflow.add_node("build_reranker", build_reranker_node)
workflow.add_node("build_chain", build_chain_node)
workflow.add_node("run_pipeline", run_pipeline_node)
workflow.add_node("show_answer", show_answer_node)

# 시작 노드
workflow.set_entry_point("ask_user_query")

# 노드 간 연결(직렬)
workflow.add_edge("ask_user_query", "build_vectorstore")
workflow.add_edge("build_vectorstore", "build_retriever")
workflow.add_edge("build_retriever", "build_reranker")
workflow.add_edge("build_reranker", "build_chain")
workflow.add_edge("build_chain", "run_pipeline")
workflow.add_edge("run_pipeline", "show_answer")
workflow.add_edge("show_answer", END)

# 그래프 컴파일
modular_rag_graph = workflow.compile()

# 시각화
visualize_graph(modular_rag_graph)

In [27]:
init_state = {
    # RAGState 형태에 맞는 필드들...
    "query": "",
    "vectorstore_built": False,
    "retriever_built": False,
    "reranker_built": False,
    "chain_built": False,
    "answer": "",
}

# config 설정
config = {"configurable": {"thread_id": "1"}}

modular_rag_graph.invoke(init_state, config=config)

[DEBUG] 입력된 질문: 년세 300 이상인 농가마트 주변 자취방 추천해줘
[DEBUG] VectorStore 빌드 완료
[DEBUG] SelfQueryRetriever 빌드 완료
[DEBUG] CrossEncoderReranker 빌드 완료
[DEBUG] RAG Chain 빌드 완료
[DEBUG] rag_pipeline 실행 완료

=== 자취방 RAG 최종 답변 ===
이름: 스마일원룸  
주소: 호서로 96-1  
가격: 년세 300  
보증금/관리비: X / 30만원  
옵션: 풀옵션  
가스 종류: 도시가스  
위치: 농가마트  

이름: 스위티빌  
주소: 호서로 112번길 10  
가격: 년세 300 / 반년세 170 / 월세 30  
보증금/관리비: X / 10 - 30만원  
옵션: 풀옵션  
가스 종류: 심야전기  
위치: 농가마트  

이름: 레몬트리  
주소: 호서로 138  
가격: 방마다 상이 (A타입 240, B타입 290, C타입 340, D타입 400)  
보증금/관리비: 연 50만원 / 50만원  
옵션: 풀옵션  
가스 종류: 산업용 일반전기  
위치: 농가마트  

더 상세한 정보는 아래 연락처로 문의해주세요.

스마일원룸 - 010-3442-3124  
스위티빌 - 010-2928-8318  
레몬트리 - 010-8217-2191  



{'query': '년세 300 이상인 농가마트 주변 자취방 추천해줘',
 'vectorstore_built': True,
 'retriever_built': True,
 'reranker_built': True,
 'chain_built': True,
 'answer': '이름: 스마일원룸  \n주소: 호서로 96-1  \n가격: 년세 300  \n보증금/관리비: X / 30만원  \n옵션: 풀옵션  \n가스 종류: 도시가스  \n위치: 농가마트  \n\n이름: 스위티빌  \n주소: 호서로 112번길 10  \n가격: 년세 300 / 반년세 170 / 월세 30  \n보증금/관리비: X / 10 - 30만원  \n옵션: 풀옵션  \n가스 종류: 심야전기  \n위치: 농가마트  \n\n이름: 레몬트리  \n주소: 호서로 138  \n가격: 방마다 상이 (A타입 240, B타입 290, C타입 340, D타입 400)  \n보증금/관리비: 연 50만원 / 50만원  \n옵션: 풀옵션  \n가스 종류: 산업용 일반전기  \n위치: 농가마트  \n\n더 상세한 정보는 아래 연락처로 문의해주세요.\n\n스마일원룸 - 010-3442-3124  \n스위티빌 - 010-2928-8318  \n레몬트리 - 010-8217-2191  '}